In [83]:
# Initial imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
import numpy as np
import hvplot.pandas
%matplotlib inline

APPLE

In [84]:
#Set path and read AAPL daily trading df.

aapl_csvpath = Path("./Resources/AAPL.csv")
aapl_df = pd.read_csv(
    aapl_csvpath, 
    index_col = 'Date',
    parse_dates = True,
    infer_datetime_format = True
)
aapl_df = aapl_df.drop(columns = 'Adj Close')
aapl_df

,Open,High,Low,Close,Volume
Date,,,,,
2018-04-02,41.660000,42.235001,41.117500,41.669998,150347200
2018-04-03,41.910000,42.187500,41.220001,42.097500,121112000
2018-04-04,41.220001,43.002499,41.192501,42.902500,138422000
2018-04-05,43.145000,43.557499,43.020000,43.200001,107732800
2018-04-06,42.742500,43.119999,42.049999,42.095001,140021200
...,...,...,...,...,...
2021-03-24,122.820000,122.900002,120.070000,120.089996,88530500
2021-03-25,119.540001,121.660004,119.000000,120.589996,98844700
2021-03-26,120.349998,121.480003,118.919998,121.209999,93958900


In [85]:
# Filter to only date and close columns
aapl_df['Close'].hvplot(
    title = 'AAPL Closing Prices',
    ylabel = 'Price in $')

:Curve   [Date]   (Close)

Dual Moving Average Crossover (DMAC) Strategy

In [86]:
# Set the variables for short window and long window periods
short_window = 7
long_window = 14

# Generate the short and long window simple moving averages (by 50 and 100 days, respectively)
aapl_df["SMA50"] = aapl_df["Close"].rolling(window=short_window).mean()
aapl_df["SMA100"] = aapl_df["Close"].rolling(window=long_window).mean()

In [87]:
# Create a column to hold the trading signal
aapl_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
aapl_df["Signal"][short_window:] = np.where(
    aapl_df["SMA50"][short_window:] > aapl_df["SMA100"][short_window:], 1.0, 0.0
)

# Review the DataFrame
aapl_df

<ipython-input-87-3885acf0d81d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl_df["Signal"][short_window:] = np.where(


,Open,High,Low,Close,Volume,SMA50,SMA100,Signal
Date,,,,,,,,
2018-04-02,41.660000,42.235001,41.117500,41.669998,150347200,NaN,NaN,0.0
2018-04-03,41.910000,42.187500,41.220001,42.097500,121112000,NaN,NaN,0.0
2018-04-04,41.220001,43.002499,41.192501,42.902500,138422000,NaN,NaN,0.0
2018-04-05,43.145000,43.557499,43.020000,43.200001,107732800,NaN,NaN,0.0
2018-04-06,42.742500,43.119999,42.049999,42.095001,140021200,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
2021-03-24,122.820000,122.900002,120.070000,120.089996,88530500,122.409999,121.621428,1.0
2021-03-25,119.540001,121.660004,119.000000,120.589996,98844700,121.698570,121.562142,1.0
2021-03-26,120.349998,121.480003,118.919998,121.209999,93958900,121.191427,121.908570,0.0


In [88]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
aapl_df["Entry/Exit"] = aapl_df["Signal"].diff()
aapl_df

,Open,High,Low,Close,Volume,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,,,,,
2018-04-02,41.660000,42.235001,41.117500,41.669998,150347200,NaN,NaN,0.0,NaN
2018-04-03,41.910000,42.187500,41.220001,42.097500,121112000,NaN,NaN,0.0,0.0
2018-04-04,41.220001,43.002499,41.192501,42.902500,138422000,NaN,NaN,0.0,0.0
2018-04-05,43.145000,43.557499,43.020000,43.200001,107732800,NaN,NaN,0.0,0.0
2018-04-06,42.742500,43.119999,42.049999,42.095001,140021200,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2021-03-24,122.820000,122.900002,120.070000,120.089996,88530500,122.409999,121.621428,1.0,0.0
2021-03-25,119.540001,121.660004,119.000000,120.589996,98844700,121.698570,121.562142,1.0,0.0
2021-03-26,120.349998,121.480003,118.919998,121.209999,93958900,121.191427,121.908570,0.0,-1.0


In [89]:
# Visualize exit position relative to close price
exit = aapl_df[aapl_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color= 'purple',
    marker = 'v',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize entry position relative to close price
entry = aapl_df[aapl_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker = '^',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize close price for the investment
security_close = aapl_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize moving averages
moving_avgs = aapl_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Apple - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [90]:
# Set the initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

# Take shares in amount of score for position where the dual moving average crossover is 1 (SMA7 is greater than SMA30)
aapl_df["Position"] = share_size * aapl_df["Signal"]

# Find the points in time where shares are bought or sold
aapl_df["Entry/Exit Position"] = aapl_df["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
aapl_df["Portfolio Holdings"] = (
    aapl_df["Close"] * aapl_df["Position"]
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
aapl_df["Portfolio Cash"] = (
    initial_capital - (aapl_df["Close"] * aapl_df["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
aapl_df["Portfolio Total"] = (
    aapl_df["Portfolio Cash"] + aapl_df["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
aapl_df["Portfolio Daily Returns"] = aapl_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
aapl_df["Portfolio Cumulative Returns"] = (
    1 + aapl_df["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
aapl_df

,Open,High,Low,Close,Volume,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,
2018-04-02,41.660000,42.235001,41.117500,41.669998,150347200,NaN,NaN,0.0,NaN,0.0,NaN,0.000,NaN,NaN,NaN,NaN
2018-04-03,41.910000,42.187500,41.220001,42.097500,121112000,NaN,NaN,0.0,0.0,0.0,0.0,0.000,100000.0000,100000.0000,NaN,NaN
2018-04-04,41.220001,43.002499,41.192501,42.902500,138422000,NaN,NaN,0.0,0.0,0.0,0.0,0.000,100000.0000,100000.0000,0.000000,0.00000
2018-04-05,43.145000,43.557499,43.020000,43.200001,107732800,NaN,NaN,0.0,0.0,0.0,0.0,0.000,100000.0000,100000.0000,0.000000,0.00000
2018-04-06,42.742500,43.119999,42.049999,42.095001,140021200,NaN,NaN,0.0,0.0,0.0,0.0,0.000,100000.0000,100000.0000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-24,122.820000,122.900002,120.070000,120.089996,88530500,122.409999,121.621428,1.0,0.0,500.0,0.0,60044.998,76829.9985,136874.9965,-0.008870,0.36875
2021-03-25,119.540001,121.660004,119.000000,120.589996,98844700,121.698570,121.562142,1.0,0.0,500.0,0.0,60294.998,76829.9985,137124.9965,0.001826,0.37125
2021-03-26,120.349998,121.480003,118.919998,121.209999,93958900,121.191427,121.908570,0.0,-1.0,0.0,-500.0,0.000,137434.9980,137434.9980,0.002261,0.37435


In [91]:
# Create the list of the metric names
metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
]

# Create a list that holds the column name
columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and columns 
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Calculate the Annualized return metric
portfolio_evaluation_df.loc['Annualized Return'] = (
    aapl_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate the Cumulative returns metric
portfolio_evaluation_df.loc['Cumulative Returns'] = aapl_df['Portfolio Cumulative Returns'][-1]

# Calculate the Annual volatility metric
portfolio_evaluation_df.loc['Annual Volatility'] = (
    aapl_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate the Sharpe ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    aapl_df['Portfolio Daily Returns'].mean() * 252) / (
    aapl_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Review the portfolio evaluation DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,0.109469
Cumulative Returns,0.37435
Annual Volatility,0.077868
Sharpe Ratio,1.405825


In [92]:
# Initialize the trade evaluation DataFrame
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

# Initialize the iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_share_price = 0.0
exit_share_price = 0.0

# Loop through the signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in aapl_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss =  exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Review the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2018-04-19,2018-04-24,500.0,43.200001,40.735001,21600.0005,20367.5005,-1232.5000
1,AAPL,2018-05-04,2018-05-25,500.0,45.957500,47.145000,22978.7500,23572.5000,593.7500
2,AAPL,2018-05-30,2018-06-18,500.0,46.875000,47.185001,23437.5000,23592.5005,155.0005
3,AAPL,2018-07-06,2018-09-14,500.0,46.992500,55.959999,23496.2500,27979.9995,4483.7495
4,AAPL,2018-09-28,2018-10-11,500.0,56.435001,53.612499,28217.5005,26806.2495,-1411.2510
5,AAPL,2019-01-16,2019-02-21,500.0,38.735001,42.764999,19367.5005,21382.4995,2014.9990
6,AAPL,2019-02-25,2019-05-10,500.0,43.557499,49.294998,21778.7495,24647.4990,2868.7495
7,AAPL,2019-06-10,2019-08-06,500.0,48.145000,49.250000,24072.5000,24625.0000,552.5000
8,AAPL,2019-08-16,2019-08-30,500.0,51.625000,52.185001,25812.5000,26092.5005,280.0005
9,AAPL,2019-09-06,2019-09-27,500.0,53.314999,54.705002,26657.4995,27352.5010,695.0015


In [93]:
trade_evaluation_df["Profit/Loss"].sum()

37434.998000000014

BANK OF AMERICA

In [12]:
#Set path and read BAC daily trading df.
bac_csvpath = Path("./Resources/BAC.csv")
bac_df = pd.read_csv(
    bac_csvpath, 
    index_col = 'Date',
    parse_dates = True,
    infer_datetime_format = True
)
bac_df = bac_df.drop(columns = 'Adj Close')
bac_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2018-04-02,29.799999,30.059999,28.750000,29.309999,86644200
2018-04-03,29.540001,29.650000,29.100000,29.590000,69698700
2018-04-04,29.000000,29.969999,28.950001,29.879999,66640800
2018-04-05,30.180000,30.549999,30.120001,30.320000,55765500
2018-04-06,30.010000,30.290001,29.370001,29.629999,79687600


In [13]:
bac_df['Close'].hvplot(
    title = 'BAC Closing Prices',
    ylabel = 'Price in $')

:Curve   [Date]   (Close)

In [14]:
# Set the variables for short window and long window periods
short_window = 50
long_window = 100

# Generate the short and long window simple moving averages (by 50 and 100 days, respectively)
bac_df["SMA50"] = bac_df["Close"].rolling(window=short_window).mean()
bac_df["SMA100"] = bac_df["Close"].rolling(window=long_window).mean()



In [15]:
# Create a column to hold the trading signal
bac_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
bac_df["Signal"][short_window:] = np.where(
    bac_df["SMA50"][short_window:] > bac_df["SMA100"][short_window:], 1.0, 0.0
)

# Review the DataFrame
bac_df

<ipython-input-15-6f3fb1353773>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bac_df["Signal"][short_window:] = np.where(


,Open,High,Low,Close,Volume,SMA50,SMA100,Signal
Date,,,,,,,,
2018-04-02,29.799999,30.059999,28.750000,29.309999,86644200,NaN,NaN,0.0
2018-04-03,29.540001,29.650000,29.100000,29.590000,69698700,NaN,NaN,0.0
2018-04-04,29.000000,29.969999,28.950001,29.879999,66640800,NaN,NaN,0.0
2018-04-05,30.180000,30.549999,30.120001,30.320000,55765500,NaN,NaN,0.0
2018-04-06,30.010000,30.290001,29.370001,29.629999,79687600,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
2021-03-24,37.209999,37.790001,36.889999,36.900002,43896800,34.4340,31.3903,1.0
2021-03-25,36.869999,37.750000,36.630001,37.660000,47517900,34.5140,31.5315,1.0
2021-03-26,38.349998,38.759998,38.080002,38.680000,66733600,34.6184,31.6813,1.0


In [16]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
bac_df["Entry/Exit"] = bac_df["Signal"].diff()
bac_df

,Open,High,Low,Close,Volume,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,,,,,
2018-04-02,29.799999,30.059999,28.750000,29.309999,86644200,NaN,NaN,0.0,NaN
2018-04-03,29.540001,29.650000,29.100000,29.590000,69698700,NaN,NaN,0.0,0.0
2018-04-04,29.000000,29.969999,28.950001,29.879999,66640800,NaN,NaN,0.0,0.0
2018-04-05,30.180000,30.549999,30.120001,30.320000,55765500,NaN,NaN,0.0,0.0
2018-04-06,30.010000,30.290001,29.370001,29.629999,79687600,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2021-03-24,37.209999,37.790001,36.889999,36.900002,43896800,34.4340,31.3903,1.0,0.0
2021-03-25,36.869999,37.750000,36.630001,37.660000,47517900,34.5140,31.5315,1.0,0.0
2021-03-26,38.349998,38.759998,38.080002,38.680000,66733600,34.6184,31.6813,1.0,0.0


In [17]:
# Visualize exit position relative to close price
exit = bac_df[bac_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color= 'purple',
    marker = 'v',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize entry position relative to close price
entry = bac_df[bac_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker = '^',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize close price for the investment
security_close = bac_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize moving averages
moving_avgs = bac_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Bank of America - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

GENERAL ELECTRIC

In [18]:
#Set path and read GE daily trading df.
ge_csvpath = Path("./Resources/GE.csv")
ge_df = pd.read_csv(
    ge_csvpath, 
    index_col = 'Date',
    parse_dates = True,
    infer_datetime_format = True
)
ge_df = ge_df.drop(columns = 'Adj Close')
ge_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2018-04-02,12.932692,13.019231,12.461538,12.615385,74970480
2018-04-03,12.615385,12.634615,12.307692,12.625000,62176504
2018-04-04,12.326923,12.788462,12.298077,12.769231,59459296
2018-04-05,12.778846,13.038462,12.663462,12.913462,55667248
2018-04-06,12.807692,12.951923,12.432692,12.557692,66004536


In [19]:
ge_df['Close'].hvplot(
    title = 'GE Closing Prices',
    ylabel = 'Price in $')

:Curve   [Date]   (Close)

In [20]:
# Set the variables for short window and long window periods
short_window = 50
long_window = 100

# Generate the short and long window simple moving averages (by 50 and 100 days, respectively)
ge_df["SMA50"] = ge_df["Close"].rolling(window=short_window).mean()
ge_df["SMA100"] = ge_df["Close"].rolling(window=long_window).mean()

In [21]:
# Create a column to hold the trading signal
ge_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
ge_df["Signal"][short_window:] = np.where(
    ge_df["SMA50"][short_window:] > ge_df["SMA100"][short_window:], 1.0, 0.0
)

# Review the DataFrame
ge_df

<ipython-input-21-c149d5d8d7c0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ge_df["Signal"][short_window:] = np.where(


,Open,High,Low,Close,Volume,SMA50,SMA100,Signal
Date,,,,,,,,
2018-04-02,12.932692,13.019231,12.461538,12.615385,74970480,NaN,NaN,0.0
2018-04-03,12.615385,12.634615,12.307692,12.625000,62176504,NaN,NaN,0.0
2018-04-04,12.326923,12.788462,12.298077,12.769231,59459296,NaN,NaN,0.0
2018-04-05,12.778846,13.038462,12.663462,12.913462,55667248,NaN,NaN,0.0
2018-04-06,12.807692,12.951923,12.432692,12.557692,66004536,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
2021-03-24,12.760000,13.020000,12.500000,12.500000,62073500,12.1874,11.1263,1.0
2021-03-25,12.400000,12.930000,12.230000,12.850000,73391900,12.2088,11.1811,1.0
2021-03-26,12.950000,13.130000,12.710000,12.990000,60452800,12.2372,11.2368,1.0


In [22]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
ge_df["Entry/Exit"] = ge_df["Signal"].diff()
ge_df

,Open,High,Low,Close,Volume,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,,,,,
2018-04-02,12.932692,13.019231,12.461538,12.615385,74970480,NaN,NaN,0.0,NaN
2018-04-03,12.615385,12.634615,12.307692,12.625000,62176504,NaN,NaN,0.0,0.0
2018-04-04,12.326923,12.788462,12.298077,12.769231,59459296,NaN,NaN,0.0,0.0
2018-04-05,12.778846,13.038462,12.663462,12.913462,55667248,NaN,NaN,0.0,0.0
2018-04-06,12.807692,12.951923,12.432692,12.557692,66004536,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2021-03-24,12.760000,13.020000,12.500000,12.500000,62073500,12.1874,11.1263,1.0,0.0
2021-03-25,12.400000,12.930000,12.230000,12.850000,73391900,12.2088,11.1811,1.0,0.0
2021-03-26,12.950000,13.130000,12.710000,12.990000,60452800,12.2372,11.2368,1.0,0.0


In [23]:
# Visualize exit position relative to close price
exit = ge_df[ge_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color= 'purple',
    marker = 'v',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize entry position relative to close price
entry = ge_df[ge_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker = '^',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize close price for the investment
security_close = ge_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize moving averages
moving_avgs = ge_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="General Electric - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

OTHER (WILL WORK ON LATER)

In [24]:
combined_df = pd.concat([aapl_df, bac_df, ge_df], axis = 1)
combined_df.tail()

,Open,High,Low,Close,Volume,SMA50,SMA100,Signal,Entry/Exit,Position,...,Entry/Exit,Open,High,Low,Close,Volume,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-24,122.820000,122.900002,120.070000,120.089996,88530500,128.9126,125.8569,1.0,0.0,500.0,...,0.0,12.76,13.02,12.50,12.50,62073500,12.1874,11.1263,1.0,0.0
2021-03-25,119.540001,121.660004,119.000000,120.589996,98844700,128.7484,125.9096,1.0,0.0,500.0,...,0.0,12.40,12.93,12.23,12.85,73391900,12.2088,11.1811,1.0,0.0
2021-03-26,120.349998,121.480003,118.919998,121.209999,93958900,128.5548,126.0331,1.0,0.0,500.0,...,0.0,12.95,13.13,12.71,12.99,60452800,12.2372,11.2368,1.0,0.0
2021-03-29,121.650002,122.580002,120.730003,121.389999,80819200,128.4044,126.1593,1.0,0.0,500.0,...,0.0,12.92,13.05,12.78,12.95,53762900,12.2630,11.2911,1.0,0.0
2021-03-30,120.110001,120.400002,118.860001,119.900002,85671900,128.2596,126.2539,1.0,0.0,500.0,...,0.0,12.98,13.36,12.90,13.30,52925100,12.3024,11.3456,1.0,0.0
